In [1]:
import math
import pandas as pd

In [2]:
def isNaN(string):
    return string!=string

In [3]:
# Read in what already exists in the AFDC data
afdc_data_raw = pd.read_csv('~/Desktop/stl/wsdot_evse/built_evse_or.csv')
afdc_data = afdc_data_raw[['id','city','state','zip','ev_level2_evse_num','ev_level1_evse_num','ev_dc_fast_num',
                          'ev_network','Y','X','open_date','ev_connector_types','ev_pricing','country']].copy()

# Add required variables for database
afdc_data['connector_code'] = -1
afdc_data['dcfc_fixed_charging_price'] = 0
afdc_data['dcfc_var_charging_price_unit'] = 0
afdc_data['dcfc_var_charging_price'] = 0
afdc_data['dcfc_fixed_parking_price'] = 0
afdc_data['dcfc_var_parking_price_unt'] = 0
afdc_data['dcfc_var_parking_price'] = 0
afdc_data['in_service'] = afdc_data_raw['status_code']

afdc_columns = ['bevse_id','city','state','zip','level2_count','level1_count','dcfc_count',
               'ev_network','latitude','longitude','open_date','ev_connector_types','ev_pricing',
               'country','connector_code','dcfc_fixed_charging_price','dcfc_var_charging_price_unit',
               'dcfc_var_charging_price','dcfc_fixed_parking_price','dcfc_var_parking_price_unt',
               'dcfc_var_parking_price','in_service']
afdc_data.columns = afdc_columns

# Modify individual variables to match the database values
new_open_date = []
new_ev_connector_codes = []
new_in_service = []

for i, row in afdc_data.iterrows():
    new_open_date.append(row['open_date'].replace('/','-'))
    if isNaN(row['ev_connector_types']):
        new_ev_connector_codes.append(0)
    elif 'CHADEMO' in row['ev_connector_types'] and 'COMBO' in row['ev_connector_types']:
        new_ev_connector_codes.append(3)
    elif 'COMBO' in row['ev_connector_types']:
        new_ev_connector_codes.append(2)
    elif 'CHADEMO' in row['ev_connector_types']:
        new_ev_connector_codes.append(1)
    elif 'TESLA' in row['ev_connector_types']:
        new_ev_connector_codes.append(4)
    else:
        new_ev_connector_codes.append(0)
        
    if row['in_service'] == 'E':
        new_in_service.append(1)
    else:
        new_in_service.append(0)

afdc_data['open_date'] = new_open_date
afdc_data['connector_code'] = new_ev_connector_codes
afdc_data['in_service'] = new_in_service

afdc_data.to_csv('~/Desktop/stl/wsdot_evse/built_evse_or_processed.csv')